# Data Collection & ETL

## Get Data (& Transform)

Going to use [ACS 5-year data](https://www.census.gov/data/developers/data-sets/acs-5year.html) - most [reliable](https://www.census.gov/programs-surveys/acs/guidance/estimates.html). Using the an existing wrapper Python package [`census`](https://github.com/datamade/census).

Get an API key from [here]("https://api.census.gov/data/key_signup.html").

In [1]:
from census import Census
import os

key = os.environ.get("census_key")
api = Census(key)

Testing the API...

In [2]:
from us import states

api.acs.state(
    ('NAME', 'B25034_010E'),
    states.PA.fips
)

[{'NAME': 'Pennsylvania', 'B25034_010E': 427320.0, 'state': '42'}]

What variables are available? See also the codings [here](https://api.census.gov/data/2019/acs/acs5/variables.html).

In [18]:
api.acs5.tables()

[{'name': 'B17015',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME',
  'variables': 'https://api.census.gov/data/2018/acs/acs5/groups/B17015.json'},
 {'name': 'B18104',
  'description': 'SEX BY AGE BY COGNITIVE DIFFICULTY',
  'variables': 'https://api.census.gov/data/2018/acs/acs5/groups/B18104.json'},
 {'name': 'B17016',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE',
  'variables': 'https://api.census.gov/data/2018/acs/acs5/groups/B17016.json'},
 {'name': 'B18105',
  'description': 'SEX BY AGE BY AMBULATORY DIFFICULTY',
  'variables': 'https://api.census.gov/data/2018/acs/acs5/groups/B18105.json'},
 {'name': 'B17017',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS BY HOUSEHOLD TYPE BY AGE OF HOUSEHOLDER',
  'variables': 'https://api.census.gov/data/2018/acs/acs5/gro

I'm interested in basic demographic variables.

In [10]:
import pandas as pd

variables = {
    'B02001_002E': 'race_white', # race - white alone
    'B02001_003E': 'race_black', # race - black or AA alone
    'B02001_004E': 'race_american', # race - American Indian and Alaska Native alone
    'B02001_005E': 'race_asian', # race - asian alone
    'B02001_006E': 'race_pi', # race - Native Hawaiian and Other Pacific Islander alone
    'B02001_007E': 'race_other', # race - some other race alone
    'B02001_008E': 'race_more', # race - 2 or more races
    'B01003_001E': 'pop_total', # total population
    'B19013_001E': 'median_income'  # median household income in the last 12 months
}
data = api.acs5.state_county_blockgroup(
    list(variables.keys()),
    states.PA.fips,  # state
    Census.ALL,  # county
    Census.ALL  # block
)
df = pd.DataFrame(data).rename(columns=variables).rename(columns={'block group': 'block_group'})
df

race_white  race_black  race_american  race_asian  race_pi  race_other  \
0         3443.0       298.0            0.0       250.0      0.0        41.0   
1           33.0      1399.0            0.0        84.0      0.0         0.0   
2          678.0       167.0           41.0         0.0      0.0       363.0   
3         1119.0        42.0            0.0         0.0      8.0         0.0   
4         1166.0         0.0            0.0         0.0      0.0         0.0   
...          ...         ...            ...         ...      ...         ...   
9735       734.0         0.0            0.0         0.0      0.0         0.0   
9736      2384.0        15.0            0.0       293.0      0.0        44.0   
9737       529.0        41.0            0.0       142.0      0.0         0.0   
9738       774.0        83.0            8.0       122.0      0.0         0.0   
9739       506.0         0.0            0.0         0.0      0.0        12.0   

      race_more  pop_total  median_income state county   tract block_group  
0          77.0     4109.0       107782.0    42    011  012105           3  
1          58.0     1574.0        50045.0    42    101  028400           1  
2           0.0     1249.0        27679.0    42    101  019200           5  
3          82.0     1251.0        66346.0    42    075  002600           1  
4           9.0     1175.0        43894.0    42    079  212300           3  
...         ...        ...            ...   ...    ...     ...         ...  
9735        0.0      734.0        95227.0    42    017  103300           2  
9736        0.0     2736.0       144450.0    42    017  100809           3  
9737        0.0      712.0       139241.0    42    017  100811           2  
9738        0.0      987.0        76806.0    42    017  100811           1  
9739       37.0      555.0        91944.0    42    017  104701           3  

[9740 rows x 13 columns]

## Load

This dataset is small, so I'm going to use Pandas' built-in SQL driver. Easier this way. For a larger dataset I might bulk add with a SQL script and a CSV file.

In [11]:
from sqlalchemy import create_engine

host = os.environ.get("ACS_HOST")
port = int(os.environ.get("ACS_PORT"))
user = os.environ.get("ACS_USER")
password = os.environ.get("ACS_PASS")
db = os.environ.get("ACS_DATABASE")
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

Replace my existing table with the ACS data. Treat this step with care!

In [12]:
from sqlalchemy.types import Integer

df.to_sql(
    'rsteed_acs_data',
    schema='acs',
    con=engine,
    if_exists='replace',
    dtype=Integer(),
    method='multi'
)

In [13]:
engine.execute("ALTER TABLE acs.rsteed_acs_data ADD PRIMARY KEY (county, tract, block_group)")

In [25]:
engine.execute("SELECT * FROM acs.rsteed_acs_data LIMIT 100").fetchall()

[(0, 3443, 298, 0, 250, 0, 41, 77, 4109, 107782, 42, 11, 12105, 3),
 (1, 33, 1399, 0, 84, 0, 0, 58, 1574, 50045, 42, 101, 28400, 1),
 (2, 678, 167, 41, 0, 0, 363, 0, 1249, 27679, 42, 101, 19200, 5),
 (3, 1119, 42, 0, 0, 8, 0, 82, 1251, 66346, 42, 75, 2600, 1),
 (4, 1166, 0, 0, 0, 0, 0, 9, 1175, 43894, 42, 79, 212300, 3),
 (5, 1011, 172, 0, 24, 0, 19, 45, 1271, -666666666, 42, 81, 500, 3),
 (6, 877, 20, 0, 0, 0, 42, 0, 939, 48359, 42, 81, 11100, 2),
 (7, 871, 4, 0, 19, 0, 2, 9, 905, 50395, 42, 103, 950102, 1),
 (8, 247, 15, 0, 31, 1, 0, 9, 303, -666666666, 42, 27, 12100, 4),
 (9, 1151, 0, 0, 48, 0, 0, 29, 1228, 129972, 42, 17, 104303, 3),
 (10, 28, 592, 0, 0, 0, 0, 21, 641, 34716, 42, 101, 6500, 5),
 (11, 1011, 0, 0, 0, 0, 0, 0, 1011, 100278, 42, 133, 22910, 3),
 (12, 1545, 459, 33, 0, 0, 0, 75, 2112, 18699, 42, 129, 800700, 1),
 (13, 696, 59, 0, 0, 0, 155, 6, 916, 106250, 42, 91, 210100, 4),
 (14, 1888, 0, 0, 13, 0, 0, 0, 1901, 57679, 42, 75, 4000, 3),
 (15, 1460, 5, 0, 0, 0, 0, 15, 14